In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightgbm as lgb

# Importing the dataset
dataset = pd.read_csv('./train_50k.csv')

FEATURE_SIZE=5


#X = dataset.iloc[:,0:4].values
X = dataset.iloc[:,0:FEATURE_SIZE-1].values
y = dataset.iloc[:,FEATURE_SIZE].values

# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 0)
x_val, x_test, y_val, y_test = train_test_split(x_val, y_val, test_size = 0.2, random_state = 0)

# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_val=sc.transform(x_val)



In [6]:
X

array([[1.00000000e+00, 8.02568218e-04, 2.18340611e-03, 1.40297836e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [5.00000000e+00, 6.34518734e-03, 5.81929912e-05, 5.43840072e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [7]:
y

array([1, 1, 1, ..., 0, 0, 0])

In [8]:
# Training
d_train = lgb.Dataset(x_train, label=y_train)
d_val = lgb.Dataset(x_val, label=y_val)
d_test=lgb.Dataset(x_test,label=y_test)


params = {
            'objective': 'binary',
            'boosting_type': 'gbdt',
#             'boosting_type': 'rf',
#            'nthread': 4,
#            'learning_rate': 0.02,  # 02,
#            'num_leaves': 15,
#            'feature_fraction': 0.9,
#            'bagging_fraction': 0.7,
#            'bagging_freq': 1,
#            'early_stopping_round':5,
#            'max_depth':2,
#            'reg_alpha': 0.041545473,
#            'reg_lambda': 0.0735294,
#            'min_split_gain': 0.0222415,
#            'min_child_weight': 60,
#            'seed': 0,
#            'verbose': -1,
            'metric': 'auc',
}




clf = lgb.train(params, d_train, 20,d_val)

#Prediction
y_pred=clf.predict(x_test)
#convert into binary values
for i in range(len(y_pred)):
    if y_pred[i]>=.5:       # setting threshold to .5
        y_pred[i]=1
    else:
        y_pred[i]=0
    
#Accuracy
from sklearn.metrics import accuracy_score
accuracy=accuracy_score(y_pred,y_test)
print(accuracy)

# AUC
from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test, y_pred))



[LightGBM] [Info] Number of positive: 15996, number of negative: 16004
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000196 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1020
[LightGBM] [Info] Number of data points in the train set: 32000, number of used features: 4
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.499875 -> initscore=-0.000500
[LightGBM] [Info] Start training from score -0.000500
[1]	valid_0's auc: 0.908583
[2]	valid_0's auc: 0.90858
[3]	valid_0's auc: 0.908456
[4]	valid_0's auc: 0.908215
[5]	valid_0's auc: 0.908196
[6]	valid_0's auc: 0.908127
[7]	valid_0's auc: 0.908137
[8]	valid_0's auc: 0.908146
[9]	valid_0's auc: 0.908453
[10]	valid_0's auc: 0.908698
[11]	valid_0's auc: 0.908709
[12]	valid_0's auc: 0.908773
[13]	valid_0's auc: 0.908776
[14]	valid_0's auc: 0.908789
[15]	valid_0's auc: 0.908804
[16]	valid_0's 

In [9]:
testdata = pd.read_csv('./test_50k.csv')
x_testing = testdata.iloc[:,0:FEATURE_SIZE-1].values
x_testing = sc.transform(x_testing)
y_pred=clf.predict(x_testing)

In [170]:
import csv
with open("kaggle_light_gbm.csv","w",newline="") as csvfile:
    writer=csv.writer(csvfile)
    writer.writerow(["Id","Predicted"])
    test_id=1
    for prediction in y_pred:
        writer.writerow([test_id,prediction])
        test_id+=1

In [132]:
# 76 % Submission on Kaggle with random sampling